In [26]:
import sys

sys.path.append("../source")

In [27]:
import json
import numpy as np
import os
import pandas as pd
import pickle
import query
import regex as re
import shutil
import sqlite3
import warnings

from collections import defaultdict
from datetime import datetime
from pandas.core.common import SettingWithCopyWarning
from pandas.tseries.offsets import MonthEnd


In [28]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [29]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

state_df = pd.read_sql_query(query.query_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [30]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

city_df = pd.read_sql_query(query.query_city_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [31]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

county_df = pd.read_sql_query(query.query_county_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [32]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

utility_df = pd.read_sql_query(query.query_utility_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [33]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_df = pd.read_sql_query(query.query_zip_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [34]:
county_df = county_df.loc[:,~county_df.columns.duplicated()]
city_df = city_df.loc[:,~city_df.columns.duplicated()]
state_df = state_df.loc[:,~state_df.columns.duplicated()]
zip_df = zip_df.loc[:,~zip_df.columns.duplicated()]
utility_df = utility_df.loc[:,~utility_df.columns.duplicated()]

keep_cols = list(
    set(zip_df.columns).intersection(
    set(utility_df.columns)).intersection(
    set(county_df.columns)).intersection(
    set(city_df.columns)).intersection(
    set(state_df.columns))
)



dsire_by_zip = pd.concat([zip_df[keep_cols],
                          utility_df[keep_cols],
                          county_df[keep_cols],
                          city_df[keep_cols],
                          state_df[keep_cols]])

In [35]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

tech_df = pd.read_sql_query(query.query_tech, conn_proj)

conn_proj.close

<function Connection.close>

In [36]:
check_list = tech_df.program_id.to_list()

In [37]:
dsire_PV = dsire_by_zip[dsire_by_zip.id.isin(check_list)]

In [38]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

sector_df = pd.read_sql_query(query.query_sector, conn_proj)

conn_proj.close

<function Connection.close>

In [39]:
dsire_PV['start_date'] = dsire_PV.start_date.fillna(dsire_PV.created_ts)
dsire_PV['end_date'] = dsire_PV.end_date.fillna(datetime.strptime(cfg["max_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
diff_df = dsire_PV.dropna(subset = ['start_date', 'zipcode']).reset_index()
diff_df['zipcode'] = diff_df.zipcode.apply(lambda n: str(int(n)).zfill(5))

diff_df = diff_df.merge(sector_df, left_on = 'id', right_on = 'program_id', how = 'left')
diff_df = diff_df[diff_df.sector_id.isin([9,23])]

In [42]:
program_map_df = diff_df[['program_type_id','zipcode','start_date','end_date','name']]
program_map_df['s_d'] = pd.to_datetime(program_map_df.start_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
program_map_df['e_d'] = pd.to_datetime(program_map_df.end_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))

program_map_df['start_month'] = pd.to_datetime(program_map_df.start_date).dt.strftime('%Y-%m')
program_map_df['start_month'] = pd.to_datetime(program_map_df['start_month']) + MonthEnd(1)

program_map_df['nb_months'] = program_map_df['start_month'].apply(lambda x: ((x - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))/np.timedelta64(1, 'M')))
program_map_df['nb_months'] = program_map_df['nb_months'].astype(int)

program_map_df["s_d"] = program_map_df.s_d.dt.days
program_map_df["e_d"] = program_map_df.e_d.dt.days
program_map_df.head()

,program_type_id,zipcode,start_date,end_date,name,s_d,e_d,start_month,nb_months
1,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
3,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
5,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
7,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
9,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67


In [43]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

program_map_df.to_sql("programs_map", conn_proj, if_exists="replace", index=False)

conn_proj.commit

<function Connection.commit>

In [20]:
def keys_exists(element, *keys):
    '''
    Check if *keys (nested) exists in `element` (dict).
    '''
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True




program_map = defaultdict(dict)
    
for a, b, c, d in zip(program_map_df.program_type_id,program_map_df.zipcode,program_map_df.s_d,program_map_df.nb_months):
    
    if keys_exists(program_map, a, b):
        program_map[a][b].append([c,d])
    else:
        program_map[a][b] = [[c,d]]

In [25]:
with open(cfg["data_dir"] + cfg["p_dates"], 'wb') as fp:
    pickle.dump(program_map, fp, protocol=pickle.HIGHEST_PROTOCOL)